In [1]:
from pytube import YouTube
from moviepy.editor import *

In [2]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import moviepy.editor as mp
import re

from src.helpers import *

In [3]:
import os
from PIL import Image
import pathlib
import csv
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
# import keras
# from keras import layers
# from keras import layers
# from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

## Read in Three CSV files

In [5]:
dataset, class_labels, true_counts = data_to_df()


In [6]:
dataset.head()
class_labels.head()
true_counts.head()

,label_id,num_rated,num_true,sound,ratio
0,/m/09x0r,10,10,Speech,1.0
1,/m/05zppz,10,9,"Male speech, man speaking",0.9
2,/m/02zsn,10,10,"Female speech, woman speaking",1.0
3,/m/0ytgt,9,9,"Child speech, kid speaking",1.0
4,/m/01h8n0,10,8,Conversation,0.8


## Clean data

In [7]:
dataset = clean_dataset(dataset, mlb)

dataset.head()

,YTID,start_seconds,end_seconds,positive_labels,recorded_labels
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""","[/m/09x0r, /t/dd00088]"
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff""",[/m/012xff]
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""","[/m/03fwl, /m/04rlf, /m/09x0r]"
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""","[/t/dd00003, /t/dd00005]"
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""","[/m/032s66, /m/073cg4]"


In [8]:
dataset.count()

YTID               22160
start_seconds      22160
end_seconds        22160
positive_labels    22160
recorded_labels    22160
dtype: int64

## Binarize entire df

### Clean true_counts data

In [9]:
true_counts = clean_true_counts(true_counts, class_labels)
true_counts.head()

,label_id,num_rated,num_true,sound,ratio
0,/m/09x0r,10,10,Speech,1.0
1,/m/05zppz,10,9,"Male speech, man speaking",0.9
2,/m/02zsn,10,10,"Female speech, woman speaking",1.0
3,/m/0ytgt,9,9,"Child speech, kid speaking",1.0
4,/m/01h8n0,10,8,Conversation,0.8


In [13]:
df = get_most_accurate(dataset, true_counts, 0.8, mlb)
df = reduce_rows(df)
df.head()

,YTID,start_seconds,end_seconds,positive_labels,/m/01280g,/m/012f08,/m/012ndj,/m/0130jx,/m/0140xf,/m/0145m,...,/t/dd00037,/t/dd00065,/t/dd00066,/t/dd00077,/t/dd00092,/t/dd00121,/t/dd00125,/t/dd00128,/t/dd00129,/t/dd00135
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,--ekDLDTUXA,30.0,40.0,"""/m/015lz1,/m/07pws3f""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.count()

YTID               18117
start_seconds      18117
end_seconds        18117
positive_labels    18117
/m/01280g          18117
                   ...  
/t/dd00121         18117
/t/dd00125         18117
/t/dd00128         18117
/t/dd00129         18117
/t/dd00135         18117
Length: 295, dtype: int64

In [216]:
b_df.head(50)

,YTID,start_seconds,end_seconds,positive_labels,/m/01280g,/m/012f08,/m/012ndj,/m/0130jx,/m/0140xf,/m/0145m,...,/t/dd00037,/t/dd00065,/t/dd00066,/t/dd00077,/t/dd00092,/t/dd00121,/t/dd00125,/t/dd00128,/t/dd00129,/t/dd00135
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,--ekDLDTUXA,30.0,40.0,"""/m/015lz1,/m/07pws3f""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,-0DLPzsiXXE,30.0,40.0,"""/m/04rlf,/m/07qwdck""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,-0DdlOuIFUI,50.0,60.0,"""/m/0130jx,/m/02jz0l,/m/0838f""",0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,-0FHUc78Gqo,30.0,40.0,"""/m/02w4v,/m/04rlf""",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,-0O3e95y4gE,100.0,110.0,"""/m/07r4wb8,/t/dd00125""",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11,-0SdAVK79lg,30.0,40.0,"""/m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rl...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
# class_labels = pd.read_csv('data/class_labels_indices.csv')
# class_labels.head()

,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation


In [55]:
# true_counts = pd.read_csv('data/qa_true_counts.csv')
# true_counts['sound'] = class_labels['display_name']

# true_counts['ratio'] = true_counts['num_true'] / true_counts['num_rated']
true_counts.sort_values('ratio', ascending=False).head(200)

,label_id,num_rated,num_true,sound,ratio
0,/m/09x0r,10,10,Speech,1.0
432,/m/0g6b5,10,10,Fireworks,1.0
191,/m/07y_7,10,10,"Violin, fiddle",1.0
426,/m/014zdl,10,10,Explosion,1.0
427,/m/032s66,10,10,"Gunshot, gunfire",1.0
...,...,...,...,...,...
430,/m/0_1c,10,8,Artillery fire,0.8
203,/m/0gy1t2s,10,8,Bicycle bell,0.8
126,/m/03vt0,10,8,Insect,0.8
199,/m/03m5k,10,8,Harp,0.8


### I only want to keep the samples with a 80% notated accuracy

In [56]:
true_counts = true_counts[true_counts['ratio'] >= 0.8]
print(true_counts.count())
true_counts.head()

label_id     221
num_rated    221
num_true     221
sound        221
ratio        221
dtype: int64


,label_id,num_rated,num_true,sound,ratio
0,/m/09x0r,10,10,Speech,1.0
1,/m/05zppz,10,9,"Male speech, man speaking",0.9
2,/m/02zsn,10,10,"Female speech, woman speaking",1.0
3,/m/0ytgt,9,9,"Child speech, kid speaking",1.0
4,/m/01h8n0,10,8,Conversation,0.8


### Now we want an array of all the most accurate notations

In [106]:
sounds_used = np.array(true_counts['label_id'])
sounds_used

array(['/m/09x0r', '/m/05zppz', '/m/02zsn', '/m/0ytgt', '/m/01h8n0',
       '/m/02qldy', '/m/0261r1', '/m/07rwj3x', '/m/04gy_2', '/t/dd00135',
       '/m/03qc9zr', '/m/02rtxlg', '/m/01j3sz', '/t/dd00001',
       '/m/07r660_', '/m/07rgt08', '/m/0463cq4', '/t/dd00002',
       '/m/07qz6j3', '/m/015lz1', '/m/0l14jd', '/t/dd00004', '/t/dd00005',
       '/m/06bxc', '/m/01w250', '/m/0lyf6', '/m/01d3sd', '/m/07pyy8b',
       '/m/01b_21', '/m/01hsr_', '/m/07pbtc8', '/m/03cczk', '/m/0939n_',
       '/m/03q5_w', '/m/02p3nc', '/m/02_nn', '/m/0k65p', '/m/025_jnm',
       '/m/0l15bq', '/m/053hz1', '/m/028ght', '/m/07rkbfh', '/m/07qfr4h',
       '/m/0jbk', '/m/0bt9lr', '/m/05tny_', '/m/07qf0zm', '/m/0ghcn6',
       '/m/01yrx', '/m/02yds9', '/m/07qrkrw', '/m/0ch8v', '/m/01xq0k1',
       '/m/07rpkh9', '/m/068zj', '/t/dd00018', '/m/03fwl', '/m/07q0h5t',
       '/m/025rv6n', '/m/09b5t', '/m/07st89h', '/m/07qn5dc', '/m/09ddx',
       '/m/07qwf61', '/m/01280g', '/m/0cdnk', '/m/015p6', '/m/020bb7',
       '

## One hot encoder the labels, find out which ones occur the most and which ones we want to focus our training on

## We need to figure out the pipeline
What I'm thinking:
- iterate through dataset
- download audio file for each video, to mp3, extract meaningful features, append to new dataframe
- if error, remove row from dataset
- clean up after each iteration

In [10]:
path = "/Users/joshbernd/Desktop/gal_notes/Capstone/CNN-Instrument-Classification/mp4"
header = make_feature_file()
i = 0
for video in dataset['YTID']:
    try:
    
        ##download youtube audio
        filename = download_video(str(video), path)
        ##convert to mp3 file
        mp3, mp4 = to_mp3(path, filename)
        ##get starting time for video
        start = int(dataset['start_seconds'][dataset['YTID'] == video])
        ##turn mp3 into spectrogram and extract features
        
        features = get_features(mp3, start)
        ##write features to dataframe
        write_to_file(features)

        os.remove(mp3)
        
        
        i += 1
        if i > 100:
            break

    

    except Exception:
        #remove row from initial dataset with invalid youtube id
        dataset = dataset[dataset['YTID'] != video]
        if i > 100:
            break
        else:
             i -= 1

    
    

chunk:   4%|▍         | 94/2262 [00:00<00:02, 915.60it/s, now=None]

MoviePy - Writing audio in /Users/joshbernd/Desktop/gal_notes/Capstone/CNN-Instrument-Classification/mp3/--PJHxphWEs.mp3


MoviePy - Done.


chunk:  39%|███▊      | 86/222 [00:00<00:00, 833.17it/s, now=None]

MoviePy - Writing audio in /Users/joshbernd/Desktop/gal_notes/Capstone/CNN-Instrument-Classification/mp3/--aE2O5G5WE.mp3


...

MoviePy - Writing audio in /Users/joshbernd/Desktop/gal_notes/Capstone/CNN-Instrument-Classification/mp3/-Jam3cjbeFQ.mp3


MoviePy - Done.


chunk:   3%|▎         | 205/6560 [00:00<00:03, 2009.90it/s, now=None]

MoviePy - Writing audio in /Users/joshbernd/Desktop/gal_notes/Capstone/CNN-Instrument-Classification/mp3/-JbPonixXS4.mp3


MoviePy - Done.


In [11]:
feature_df = pd.read_csv('dataset.csv')
feature_df.head()

,YTID,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,--PJHxphWEs,0.144023,0.054509,3083.525008,2727.951400,6136.962891,0.000488,-677.725464,-60.321907,-1.675254,...,13.621056,65.283966,22.037888,-34.077576,1.344032,20.719837,-14.135757,39.440418,57.690269,-45.903584
1,--aE2O5G5WE,0.146669,0.021313,2056.899989,3176.120465,4909.570312,0.000000,-777.996643,-49.890053,13.396999,...,19.317492,51.159569,19.382998,-14.735193,-7.327859,12.584110,6.626073,29.135651,46.241631,-31.901894
2,--aaILOrkII,0.139654,0.399394,480.337467,1242.651527,1227.392578,0.000000,-516.471558,-17.606678,-2.516232,...,17.650810,48.678459,19.960388,-8.039909,-14.162983,9.131400,13.745262,32.253983,34.012314,-23.337450
3,--cB2ZVjpnA,0.110790,0.024786,2349.680827,1963.629772,3671.411133,0.000977,-735.875793,-43.371239,-12.871243,...,19.297230,63.482571,23.846874,-34.264587,-3.065814,15.538205,-11.578226,44.499367,48.511829,-40.622833
4,--ekDLDTUXA,0.138681,0.165942,1829.442944,2935.817842,4888.037109,0.000000,-582.247437,-39.230446,1.131112,...,22.324093,48.208324,36.402683,-29.619797,-3.743885,4.963671,13.029894,30.473604,50.506432,-39.929340


In [13]:
feature_df.head(131)

,YTID,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,--PJHxphWEs,0.144023,0.054509,3083.525008,2727.951400,6136.962891,0.000488,-677.725464,-60.321907,-1.675254,...,13.621056,65.283966,22.037888,-34.077576,1.344032,20.719837,-14.135757,39.440418,57.690269,-45.903584
1,--aE2O5G5WE,0.146669,0.021313,2056.899989,3176.120465,4909.570312,0.000000,-777.996643,-49.890053,13.396999,...,19.317492,51.159569,19.382998,-14.735193,-7.327859,12.584110,6.626073,29.135651,46.241631,-31.901894
2,--aaILOrkII,0.139654,0.399394,480.337467,1242.651527,1227.392578,0.000000,-516.471558,-17.606678,-2.516232,...,17.650810,48.678459,19.960388,-8.039909,-14.162983,9.131400,13.745262,32.253983,34.012314,-23.337450
3,--cB2ZVjpnA,0.110790,0.024786,2349.680827,1963.629772,3671.411133,0.000977,-735.875793,-43.371239,-12.871243,...,19.297230,63.482571,23.846874,-34.264587,-3.065814,15.538205,-11.578226,44.499367,48.511829,-40.622833
4,--ekDLDTUXA,0.138681,0.165942,1829.442944,2935.817842,4888.037109,0.000000,-582.247437,-39.230446,1.131112,...,22.324093,48.208324,36.402683,-29.619797,-3.743885,4.963671,13.029894,30.473604,50.506432,-39.929340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,-JKLmqDk9p8,0.192781,0.004820,4832.315379,3669.252886,9797.607422,0.002441,-914.773804,-65.093506,13.926597,...,12.904678,58.368774,29.181709,-23.927414,-15.933074,15.994800,2.668251,33.912552,52.804276,-44.254318
127,-JOKwS64mRU,0.489950,0.043773,203.299598,996.396862,10.766602,0.000000,-755.765076,-20.068527,10.470110,...,14.668190,32.084675,-1.794517,4.894401,10.609842,14.049875,1.249871,13.016512,29.078253,-4.659967
128,-JVWfFPBl84,0.135420,0.021622,1423.281525,2442.617707,2519.384766,0.000000,-790.405884,-47.060619,10.491171,...,16.250748,47.837238,2.965533,-8.182658,7.010826,18.803110,-5.540838,20.291389,46.918987,-23.676163
129,-Jam3cjbeFQ,0.111342,0.069059,2645.576128,2703.945140,4909.570312,0.000977,-671.135559,-57.651184,-0.991896,...,22.715660,66.719612,17.568981,-26.835751,-1.164851,15.078834,-4.106844,35.412018,58.241287,-41.305882


In [14]:
dataset.head(131)

,YTID,start_seconds,end_seconds,positive_labels,recorded_labels
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""","[/m/09x0r, /t/dd00088]"
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""","[/m/03fwl, /m/04rlf, /m/09x0r]"
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""","[/m/032s66, /m/073cg4]"
5,--cB2ZVjpnA,30.0,40.0,"""/m/01y3hg""",[/m/01y3hg]
6,--ekDLDTUXA,30.0,40.0,"""/m/015lz1,/m/07pws3f""","[/m/015lz1, /m/07pws3f]"
...,...,...,...,...,...
156,-JKLmqDk9p8,490.0,500.0,"""/m/06mb1,/m/07r10fb,/m/0jb2l,/m/0ngt1,/t/dd00...","[/m/06mb1, /m/07r10fb, /m/0jb2l, /m/0ngt1, /t/..."
157,-JOKwS64mRU,150.0,160.0,"""/m/019jd,/m/03m9d0z,/m/06q74,/t/dd00092""","[/m/019jd, /m/03m9d0z, /m/06q74, /t/dd00092]"
158,-JVWfFPBl84,330.0,340.0,"""/m/0l14t7""",[/m/0l14t7]
159,-Jam3cjbeFQ,30.0,40.0,"""/m/07pws3f,/m/09x0r""","[/m/07pws3f, /m/09x0r]"


In [ ]:
a = int(sr/2)
b = int(sr*(3/2))
fig, ax = plt.subplots(figsize = (25, 10))
snippet = data[sr*5:sr*6]
ax.plot(snippet)

## Save audio as spectrogram.png

In [ ]:
def get_audio(file, start, stop):
    '''
    parameters:
        file: str, location of audio filein mp3 audio file
        start: beggining of clip in seconds
        stop: end of clip in seconds
    returns:
        tuple
        data: numpy array
        sr: int
    '''
    data, sr = librosa.load(file, mono=True)
    data = data[sr*start:sr*stop]
    return (data, sr)

def get_spec(lib_file):
    cmap = plt.get_cmap('inferno')
    plt.figure(figsize=(8,8))
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, 
                 noverlap=128, cmap=cmap, 
                 sides='default', mode='default', 
                 scale='dB');
    plt.axis('off')
    plt.savefig(f'/Users/joshbernd/Desktop/gal_notes/Capstone/Capstone_2/img/{filename[:-3].replace(".", "")}.png')
    plt.clf()
    
    
def trim(y, sr, start, stop):
    y = y[sr*start:sr*stop]


    
    
data = get_audio('/Users/joshbernd/Desktop/gal_notes/Capstone/Capstone_2/downloaded_videos/Vince Guaraldi Trio - Linus And Lucy.mp3', 5, 15)[0]    
get_spec(data)

## 5. Extracting features from Spectrogram: 
1 - Mel-frequency cepstral coefficients (MFCC) 

2 - Spectral Centroid

3 - Zero Crossing Rate

4 - Chroma Frequencies

5 - Spectral Roll-off.